In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings(action='ignore')
import emoji
import contractions
import re
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [2]:
#Here we read the data, drop columns we don't use, and drop reviews longer than 150 words.
dataset=pd.read_csv('stcpay_reviews.csv')
dataset

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,cbd3f291-3669-4221-b030-c2d5b65d0b81,Sakib Ahmed,https://play-lh.googleusercontent.com/a-/AD_cM...,"Very Poor Application, Specially For Internati...",1,0,1.10.30,2023-06-12 09:31:27,NaN,NaN,1.10.30
1,9e01964e-cfaf-48f9-a727-9f746d402208,M A N i A C,https://play-lh.googleusercontent.com/a-/AD_cM...,Not receive otp 🤷🏻,1,0,NaN,2023-06-12 08:50:26,NaN,NaN,NaN
2,04536120-95a7-4dcf-98ae-027979ea060b,Santosh Thakur,https://play-lh.googleusercontent.com/a-/AD_cM...,Very good application 👍,5,0,1.10.30,2023-06-12 07:29:48,NaN,NaN,1.10.30
3,8269172a-7f9b-4854-85e7-372bd0170e89,ابو يوسف,https://play-lh.googleusercontent.com/a-/AD_cM...,تطبيق سئ وخدمة عملاء أسوأ منها .....للأسف حاجز...,1,0,1.10.30,2023-06-12 03:28:26,هلا والله ابو يوسف، ياليت تتواصل مع الدعم وتعط...,2022-10-09 05:51:05,1.10.30
4,4ab08bf2-5a62-46a9-a95c-d4d016f203f3,zeeshan shoeb,https://play-lh.googleusercontent.com/a/AAcHTt...,nice,1,0,NaN,2023-06-12 02:36:11,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
20615,f71ef9fc-dc44-468d-bfbc-77ac811372d0,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Not working,1,4,1.0.5.9,2018-09-20 23:37:17,"Hi,\n\nWe apologize for the problem. Please se...",2018-10-08 22:23:36,1.0.5.9
20616,175778c4-888b-4f3a-9c8d-84355655567b,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Registration by innovation only..,2,1,1.0.5.7,2018-09-11 00:51:32,"Hi,\n\nIt will be available to everyone in the...",2018-10-08 22:18:20,1.0.5.7
20617,52126110-12e3-4acd-a1a1-e2acf7d9e832,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,It is what i need,5,2,1.0.5.6,2018-09-09 11:24:37,"Thank you for your support, hope you enjoy usi...",2018-10-08 03:56:42,1.0.5.6
20618,5c50ee53-cad6-4441-8281-e8781376f019,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,"Wonderful idea, also loved the app",5,1,1.0.1.9,2018-06-03 06:35:23,Thank you,2018-10-08 03:56:02,1.0.1.9


In [3]:
data_in = dataset.copy()
data_in = data_in.drop_duplicates(subset="content")  # Drop dupes in Reviews
data_in.drop(["reviewId", "userName",
             "userImage","score","thumbsUpCount","reviewCreatedVersion","replyContent","repliedAt"], axis=1, inplace=True)
data_in["Length"] = data_in["content"].str.split(" ").str.len()
data_in = data_in[data_in.Length < 150]
data_in.drop(["Length"], axis=1, inplace=True)
data = data_in
data

,content,at,appVersion
0,"Very Poor Application, Specially For Internati...",2023-06-12 09:31:27,1.10.30
1,Not receive otp 🤷🏻,2023-06-12 08:50:26,NaN
2,Very good application 👍,2023-06-12 07:29:48,1.10.30
3,تطبيق سئ وخدمة عملاء أسوأ منها .....للأسف حاجز...,2023-06-12 03:28:26,1.10.30
4,nice,2023-06-12 02:36:11,NaN
...,...,...,...
20613,Needs invitation to work,2018-09-25 13:53:54,1.0.5.9
20614,it doesn't work !!!,2018-09-23 09:37:39,1.0.5.9
20616,Registration by innovation only..,2018-09-11 00:51:32,1.0.5.7
20617,It is what i need,2018-09-09 11:24:37,1.0.5.6


In [4]:
def clean_text(text):
    text = emoji.demojize(text)  # fix emojis
    text = contractions.fix(text)  # fix contractions
    text = re.sub(r'[^\x00-\x7f]', r'', text)  # remove strange fonts
    text = re.sub(r"\d+", "number", text)  # replace numbers with "number"
    text = re.sub(r'[^\w\s]', '', text)  # remove non-alphanumeric chars
    text = text.replace('_', ' ')  # replace underscores with space
    text = text.strip()  # strip extra spaces
    return text


In [5]:
data["content"] = data["content"].apply(clean_text)
data

,content,at,appVersion
0,Very Poor Application Specially For Internatio...,2023-06-12 09:31:27,1.10.30
1,Not receive otp person shrugging light skin tone,2023-06-12 08:50:26,NaN
2,Very good application thumbs up,2023-06-12 07:29:48,1.10.30
3,number August stc pay nu...,2023-06-12 03:28:26,1.10.30
4,nice,2023-06-12 02:36:11,NaN
...,...,...,...
20613,Needs invitation to work,2018-09-25 13:53:54,1.0.5.9
20614,it does not work,2018-09-23 09:37:39,1.0.5.9
20616,Registration by innovation only,2018-09-11 00:51:32,1.0.5.7
20617,It is what i need,2018-09-09 11:24:37,1.0.5.6


In [6]:
#Checking for null values
data.isnull().sum()

content          0
at               0
appVersion    2574
dtype: int64

In [7]:
!pip install textstat
!pip install nltk


In [8]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import RegexpTokenizer
from textstat import flesch_reading_ease

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon')
nltk.download('words')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [9]:
sid = SentimentIntensityAnalyzer()
words = set(nltk.corpus.words.words())


In [10]:
positive_scores = []
negative_scores = []
neutral_scores = []
compound_scores = []


    


In [11]:
for index, row in data.iterrows():
    contents = row['content']
    
    sentiment_scores = sid.polarity_scores(contents)
    
    positive_score = sentiment_scores['pos']
    negative_score = sentiment_scores['neg']
    neutral_score = sentiment_scores['neu']
    compound_score = sentiment_scores['compound']
    
    positive_scores.append(positive_score * 100)
    negative_scores.append(negative_score * 100)
    neutral_scores.append(neutral_score * 100)
    compound_scores.append(compound_score * 100)

In [12]:
data['POSITIVE SCORE'] = positive_scores
data['NEGATIVE SCORE'] = negative_scores
data['NEUTRAL SCORE'] = neutral_scores
data['COMPOUND SCORE'] = compound_scores


data

,content,at,appVersion,POSITIVE SCORE,NEGATIVE SCORE,NEUTRAL SCORE,COMPOUND SCORE
0,Very Poor Application Specially For Internatio...,2023-06-12 09:31:27,1.10.30,13.2,18.0,68.8,-22.47
1,Not receive otp person shrugging light skin tone,2023-06-12 08:50:26,NaN,0.0,0.0,100.0,0.00
2,Very good application thumbs up,2023-06-12 07:29:48,1.10.30,44.4,0.0,55.6,49.27
3,number August stc pay nu...,2023-06-12 03:28:26,1.10.30,37.1,20.0,42.9,5.16
4,nice,2023-06-12 02:36:11,NaN,100.0,0.0,0.0,42.15
...,...,...,...,...,...,...,...
20613,Needs invitation to work,2018-09-25 13:53:54,1.0.5.9,0.0,0.0,100.0,0.00
20614,it does not work,2018-09-23 09:37:39,1.0.5.9,0.0,0.0,100.0,0.00
20616,Registration by innovation only,2018-09-11 00:51:32,1.0.5.7,46.4,0.0,53.6,38.18
20617,It is what i need,2018-09-09 11:24:37,1.0.5.6,0.0,0.0,100.0,0.00


In [13]:
positive_threshold = 0.2
negative_threshold = -0.2

data['Sentiment']= ''

for index, row in data.iterrows():
    compound_score = row['COMPOUND SCORE']
    
    if compound_score > positive_threshold:
        data.at[index, 'Sentiment'] = 'Positive'
    elif compound_score < negative_threshold:
            data.at[index, 'Sentiment'] = 'Negative'
    else:
                data.at[index, 'Sentiment'] = 'Neutral'

In [14]:
data.head(50)

,content,at,appVersion,POSITIVE SCORE,NEGATIVE SCORE,NEUTRAL SCORE,COMPOUND SCORE,Sentiment
0,Very Poor Application Specially For Internatio...,2023-06-12 09:31:27,1.10.30,13.2,18.0,68.8,-22.47,Negative
1,Not receive otp person shrugging light skin tone,2023-06-12 08:50:26,NaN,0.0,0.0,100.0,0.00,Neutral
2,Very good application thumbs up,2023-06-12 07:29:48,1.10.30,44.4,0.0,55.6,49.27,Positive
3,number August stc pay nu...,2023-06-12 03:28:26,1.10.30,37.1,20.0,42.9,5.16,Positive
4,nice,2023-06-12 02:36:11,NaN,100.0,0.0,0.0,42.15,Positive
5,Perfect,2023-06-11 13:06:56,NaN,100.0,0.0,0.0,57.19,Positive
6,Work for number seconds then crashes every time,2023-06-11 12:46:18,NaN,15.7,0.0,84.3,7.72,Positive
7,Nicc app mony tarnsfar,2023-06-11 11:51:02,NaN,0.0,0.0,100.0,0.00,Neutral
8,Good workbackhand index pointing rightred heart,2023-06-11 06:21:13,1.10.30,36.7,0.0,63.3,44.04,Positive
9,Good,2023-06-10 20:52:56,1.10.30,100.0,0.0,0.0,44.04,Positive


In [15]:
data['Sentiment'].value_counts()

Positive    7084
Negative    3446
Neutral     3034
Name: Sentiment, dtype: int64

In [16]:
data.to_csv('SentimentSTCPAY.csv', index=False)